In [1]:
import pandas as pd

In [3]:
csv_input = pd.DataFrame(pd.read_csv(filepath_or_buffer="C:/Users/Winte/Downloads/Test_for_the_seminar_apply/Book2.csv", encoding="ms932", sep=","))
#print(csv_input) non dataframe object

In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Winte\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
import string
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet
import numpy as np
nltk.download("wordnet")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Winte\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [38]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(csv_input, random_state = 9999 )


In [39]:
train_df_bert = pd.DataFrame({
    'id':range(len(train_df)),
    'label':train_df["Recommended IND"],
    'alpha':['a']*train_df.shape[0],
    'text': train_df["Review Text"]})

train_df_bert.head()

test_df_bert = pd.DataFrame({
    'id':range(len(test_df)),
    'label':test_df["Recommended IND"],
    'alpha':['a']*test_df.shape[0],
    'text': test_df["Review Text"]})

#test_df_bert.head()

In [41]:
train_df_bert.to_csv('Bert/data/train.tsv', sep='\t', index=False, header=False)
test_df_bert.to_csv('Bert/data/dev.tsv', sep='\t', index=False, header=False)

In [6]:
df_t = pd.DataFrame(csv_input['Review Text'])
df_ind = pd.DataFrame(csv_input['Recommended IND'])
#print(df_t.head(10))



                                         Review Text
0  Absolutely wonderful - silky and sexy and comf...
1  Love this dress! it sooo pretty. i happened to...
2  I had such high hopes for this dress and reall...
3  I love, love, love this jumpsuit. it fun, flir...
4  This shirt is very flattering to all due to th...
5  I love tracy reese dresses, but this one is no...
6  I aded this in my basket at hte last mintue to...
7  I ordered this in carbon for store pick up, an...
8  I love this dress. i usually get an xs but it ...
9  I'm 5"5' and 125 lbs. i ordered the s petite t...


In [7]:
#removing punctuations
string.punctuation=('!"#$%&()*+,-./:;<=>?@[\]^_`{|}~')

def remove_punctuation(text):
    no_punct = "".join([i for i in text if i not in string.punctuation])
    return no_punct


df_t["Review Text"] = df_t["Review Text"].apply(lambda x: remove_punctuation(x))

#df_t.head()

In [8]:
#TOKENIZING
tokenizer = RegexpTokenizer('\w+|\$[\d\.]+')
df_t["Review Text"] = df_t["Review Text"].apply(lambda x: tokenizer.tokenize(x.lower()))
print(df_t.head(10))

                                         Review Text
0  [absolutely, wonderful, silky, and, sexy, and,...
1  [love, this, dress, it, sooo, pretty, i, happe...
2  [i, had, such, high, hopes, for, this, dress, ...
3  [i, love, love, love, this, jumpsuit, it, fun,...
4  [this, shirt, is, very, flattering, to, all, d...
5  [i, love, tracy, reese, dresses, but, this, on...
6  [i, aded, this, in, my, basket, at, hte, last,...
7  [i, ordered, this, in, carbon, for, store, pic...
8  [i, love, this, dress, i, usually, get, an, xs...
9  [i, m, 55, and, 125, lbs, i, ordered, the, s, ...


In [9]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()

df_t["Review Text"] = df_t["Review Text"].apply(lambda x: 
                                                [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in x])

df_t.head(10)

,Review Text
0,"[absolutely, wonderful, silky, and, sexy, and,..."
1,"[love, this, dress, it, sooo, pretty, i, happe..."
2,"[i, have, such, high, hope, for, this, dress, ..."
3,"[i, love, love, love, this, jumpsuit, it, fun,..."
4,"[this, shirt, be, very, flatter, to, all, due,..."
5,"[i, love, tracy, reese, dress, but, this, one,..."
6,"[i, aded, this, in, my, basket, at, hte, last,..."
7,"[i, order, this, in, carbon, for, store, pick,..."
8,"[i, love, this, dress, i, usually, get, an, x,..."
9,"[i, m, 55, and, 125, lb, i, order, the, s, pet..."


In [15]:
#REMOVE STOPWORDS
##IF SUCCESSED CHANGED ALL DF NAME BELOW

from nltk.corpus import stopwords
nltk.download("stopwords")
stop_words= stopwords.words("english")

stop_words = [x for x in stop_words if x not in ["not","no"]]

#print(stop_words)

df_t["Review Text"] = df_t["Review Text"].apply(lambda x: [item for item in x if item not in stop_words])
df_t["Recommended IND"]  = df_ind
print(df_t.head())
df_t.to_csv("processed_data.csv")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Winte\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


                                         Review Text  Recommended IND
0  [absolutely, wonderful, silky, sexy, comfortable]                1
1  [love, dress, sooo, pretty, happen, find, stor...                1
2  [high, hope, dress, really, want, work, initia...                0
3  [love, love, love, jumpsuit, fun, flirty, fabu...                1
4  [shirt, flatter, due, adjustable, front, tie, ...                1


In [37]:
train_df, test_df = train_test_split(df_t, random_state = 9999 )

train_df.to_csv('data/train.csv', index=False, header=False)
test_df.to_csv('data/test.csv', index=False, header=False)